# Initial experiments (remote)

* Some rudimentary experiments with deeper architectures, which appear to overfit badly (need to learn more)
* It may be that even xresnet50 isn't properly pretrained, as it underperforms compared to resnet50
* Main goal of having a Kaggle setup was achieved

In [3]:
import experiments

from fastai.vision.all import *
import mlflow

In [4]:
INPUT_ROOT = "/kaggle/input"
OUTPUT_ROOT = "/kaggle/working/deep-fish"

In [5]:
!mkdir -p deep-fish

In [6]:
# Vacuum compacts the file -- it's essentially a no-op to create the file if it doesn't exist
!sqlite3 deep-fish/mlruns.db "VACUUM;"

In [7]:
mlflow.set_tracking_uri(f"sqlite:///{OUTPUT_ROOT}/mlruns.db")
mlflow.set_registry_uri(f"file://{OUTPUT_ROOT}/mlruns")
mlflow.set_experiment("initial-rls-species-100-min-images-4")

In [8]:
create_reproducible_learner = partial(
    experiments.create_reproducible_learner, dataset_path=Path(f"{INPUT_ROOT}/rls-species-100-min-images-4")
)

In [7]:
create_reproducible_learner(resnet18, dls_kwargs=dict(verbose=True)).dls.show_batch()

In [11]:
with mlflow.start_run(run_name="xresnet50"):
    learner = create_reproducible_learner(xresnet50)
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    learner.fine_tune(20)
    mlflow.log_metrics(dict(zip(learner.recorder.metric_names[1:-1], learner.recorder.log[1:-1])))

In [13]:
with mlflow.start_run(run_name="xresnet50-more-fine-tuning"):
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("start_learner", "initial")
    learner.fine_tune(100)
    mlflow.log_metrics(dict(zip(learner.recorder.metric_names[1:-1], learner.recorder.log[1:-1])))

In [8]:
with mlflow.start_run(run_name="xresnet50-mixup-label-smoothing"):
    learner = create_reproducible_learner(
        xresnet50,
        learner_kwargs=dict(loss_func=LabelSmoothingCrossEntropy(), cbs=[experiments.MLflowCallback(), MixUp()]),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("loss_func", "LabelSmoothingCrossEntropy()")
    mlflow.log_param("cbs", "MixUp()")

    learner.fine_tune(120)

    mlflow.log_metrics(experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0))

In [9]:
with mlflow.start_run(run_name="resnet50-mixup-label-smoothing"):
    learner = create_reproducible_learner(
        resnet50,
        learner_kwargs=dict(loss_func=LabelSmoothingCrossEntropy(), cbs=[experiments.MLflowCallback(), MixUp()]),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("loss_func", "LabelSmoothingCrossEntropy()")
    mlflow.log_param("cbs", "MixUp()")

    learner.fine_tune(120)

    mlflow.log_metrics(experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0))